In [2]:
KK0="文档起始位"
KK1="文档活动位"
KK2="文档终止位"

# 类和面向对象编程

In [3]:
import brtoz.brmagic

## class语句

类中的`方法`, `变量`, `属性`统称为**类成员**;    
类(class) 是方法(method), 类变量(class variable), 和属性(property)的集合;  
@类是设计对象的一种蓝图, 框架, 类本身自带命名空间;  

类的创建:   
self表示**类的实例**;  
类中定义的函数称为**实例方法**, 实例方法对类实例进行操作;

In [4]:
class Account(object):
    num_accounts = 0
    def __init__(self, name, balance):  #balance金额
        self.name = name
        self.balance = balance
        Account.num_accounts += 1       #self.num_accounts += 1      
    def __del__(self):
        Account.num_accounts -= 1
    def deposit(self, amt):             #存款
        self.balance = self.balance + amt
    def withdraw(self, amt):            #取款
        self.balance = self.balance - amt
    def inquiry(self):                  #查询余额
        return self.balance

访问类中的成员就是访问**类命名空间**中的变量;    

In [5]:
%%pass
Account.num_accounts                 # 用户数
Account.__init__                     # 执行类的初始化
Account.__del__                      # 删除用户的函数
Account.deposit                      # 存款函数
Account.withdraw                     # 取款函数
Account.inquiry                      # 查询函数

## 类实例

通过类创建的对象叫做**类实例**;    
创建类实例时, 类型名所带的参数其实都是方法`__init__`()中的参数;

类实例化的过程就是执行类的初始化过程:     
`self.name = name`: 将`name`参数放入类对象的命名空间;    
`Account.num_accounts += 1`: 每执行一次实例化过程, 让类变量`num_accounts`的值就增加1;

类的实例化

In [6]:
a = Account('ura', 100)
b = Account('urb', 100)

参与属性访问的属性既可以来自于`实例`也可以来自`实例所属的类`;   
**属性访问的查找顺序**: 先在实例中查找该属性, 实例中没有的话再在其所属的类中查找;

In [7]:
a.deposit(100)
a.inquiry()

200

In [8]:
b.withdraw(100)
b.inquiry()

0

## 作用域规则

**可以通过slef或类名引用类的上下文成员**, 而不能直接使用成员名;

In [9]:
class Foo(object):
    def bar(self):
        print('bar')
    def spam(self):
        try:
            bar(self)  #NameError: name 'bar' is not defined
        except NameError as e:
            print("NameError: ", e)
    def spam2(self):
        self.bar() #correct Foo.bar(self) #also crrect

In [10]:
foo = Foo()

In [11]:
foo.spam()

NameError:  name 'bar' is not defined


In [12]:
foo.spam2()

bar


## 继承

继承: 原始类相新类传递`蓝图`的过程;   
原始类: `基类, 超类, 父类`;   新类: `派生类, 子类`;  
派生类可以重新定义超类的属性并添加自己的属性;

加强版的Account类

In [13]:
import random
class EvilAccount(Account):
    def inquiry(self): #有四分之一的概率
        if random.randint(0,4)==1: #虚假剩余额, 高估自己的剩余额
            return self.balance * 1.10
        else:
            return self.balance

In [14]:
c = EvilAccount("urc", 100)
c.deposit(100)
c.inquiry() #大部分情况输出200, 少部分情况输出220

200

有了继承, **属性访问的查找顺序**也会增添新的`路径`:    
先在实例中查找该属性, 实例中没有的话再在其所属的类中查找, 如果所属的类中也没有该属性, 继续查找该类的超类, 超类的超类, 一直查找到根object为止;

子类可以定义自己的`__init__`函数, 从而向实例添加新属性

In [15]:
class EvilAccount(Account):
    def __init__(self,name,balance,evilfactor):
        Account.__init__(self,name,balance) #初始化Account, 可以不带父类的参数
        self.evilfactor = evilfactor
    def inquiry(self):
        if random.randint(0,4)==1: #虚假存款, 高估自己的存款
            return self.balance * evilfactor #也可以使用self.evilfactor
        else:
            return self.balance

> 派生类定义`__init__`时不会自动调用该父类的`__init__`方法;  
派生类调用父类的`__init__`方法;  
* 带参数调用: `Account.__init__(self,name,balance)`
* 如果不知道基类是否定义了`__init__`, 可以在不提供任何参数的情况下调用它: `Account.__init__`();  

派生类将重新实现方法, 但是还想调用原始的方法;   
在新类中可以显示的调用父类中的原始方法, 将self作为第一个参数传递即可;  

In [16]:
class MoreEvilAccount(EvilAccount):
    def deposit(self,amount):
        self.withdraw(5.00)   #手续费
        EvilAccount.deposit(self.amount) #现在进行存款

注意: EvilAccount其实并没有定义 deposit, 只是继承了 Account 的 deposit, 为了更明确的表名 deposit 的来源, 可以使用 **super()函数**: 希望调用以前的实现, 而不管它是在哪个基类中定义的;

In [17]:
# 将上例改为
class MoreEvilAccount(EvilAccount):
    def deposit(self,amount):
        self.withdraw(5.00)   
        super().deposit(amount) #Python3可简写, 表名 deposit 可以来源该于该类的任何父类
        #super(MoreEvilAccount, self).deposit(amount) 

python支持多重继承, 因此`class KK(c1,c2,..,cn):`这种写法成立;

In [18]:
class DepositCharge(object):
    fee=5.00
    def deposit_fee(self):
        self.withdraw(self.fee)

class WithdrawCharge(object):
    fee=2.50
    def withdraw_fee(self):
        self.withdraw(self.fee)

# 使用多重继承
class MostEvilAccount(EvilAccount, DepositCharge, WithdrawCharge):
    def deposit(self,amt):
        self.deposit_fee()
        super(MostEvilAccount,self).deposit(amt)
    def withdraw(self,amt):
        self.withdraw_fee()
        super(MostEvilAccount,self),withdraw(amt)        

使用多重继承之后, 属性的解析将变得非常复杂, 因为可以使用很多搜索路径来绑定属性;

In [19]:
%%pass 运行导致程序终止?
d = MostEvilAccount("urd", 100, 1.10)
d.deposit_fee()

基类的排列顺序

In [20]:
MostEvilAccount.__mro__

(__main__.MostEvilAccount,
 __main__.EvilAccount,
 __main__.Account,
 __main__.DepositCharge,
 __main__.WithdrawCharge,
 object)

类的层次结构

In [24]:
try:
    class X(object): pass #先定义X, 先检查类X
    class Y(X): pass      #Y继承于X, Y更特殊, 先检查Y
    class Z(X,Y): pass
except TypeError as e:
    print("无法创建一致的方法解析顺序")

无法创建一致的方法解析顺序


## 多态动态绑定和鸭子类型

**动态绑定**(在继承背景下, 也被称为**多态性**): 可以在不考虑实例类型的情况下使用实例;  
**鸭子类型**: 看起来像, 叫起来像, 走起路来像的东西就是鸭子, 而不管它是不是真正的鸭子;  

如何实现多态性:  
* 继承;
* 外观和行为像, 但是没有继承关系, 保持`程序组件的松散耦合`;

## 静态方法和类方法

在类定义中, 所有函数的操作对象都假定为调用它的实例(实例本身总是作为第一个参数传递到其所召唤的方法中); 在类命名空间中还存在其他的方法, 不需要调用实例本身;  
**静态方法**: 位于类命名空间中的普通函数, 它不会对任何实例类型进行操作;  
**类方法**: 类方法是操作类本身的方法;

`@staticmethod`装饰器用于定义静态方法

In [26]:
class Foo(object):
    @staticmethod
    def add(x,y):
        return x + y

要调用静态方法, 就像调用普通函数一样简单, 但是要添加所在类的类名作为前缀;

In [28]:
Foo.add(3,4)

7

`@classmethod`装饰器用于定义类方法, 类方法用`cls`表示类本身;

In [29]:
class Times(object):
    factor = 1
    @classmethod
    def mul(cls,x):
        return cls.factor*x
class TwoTimes(Times):
    factor = 2

In [31]:
TwoTimes.mul(4) # 类TwoTimes调用mul, cls表示的就是类TwoTimes

8

类方法的用途: 

## 特性

**特性**是对类变量信息的一种二次处理(综合或者概括), 可以使属性的指向性更加清晰和明确;

In [40]:
import math
class Circle(object):
    def __init__(self,radius):
        self.radius = radius
    #定义Circle的一些附加特性
    @property
    def area(self):
        return round(math.pi * self.radius**2, 3)
    @property
    def perimeter(self):
        return round(2*math.pi * self.radius, 3)    

In [45]:
c = Circle(1) #area是特性, 调用时不接`()`
%C c.radius; c.area; c.perimeter

c.radius  c.area  c.perimeter
--------  ------  -----------
1         3.142   6.283      


In [44]:
try:
    c.area = 1
except AttributeError as e:
    print(e)

c.radius  c.area  c.perimeter
--------  ------  -----------
1         3.142   6.283      
can't set attribute


方法本身是被隐性的作为一类特性处理的, f.spam返回一个**绑定方法**对象

In [50]:
class Foo(object):
    def __init__(self, name):
        self.name = name
    def spam(self, x):
        print(self.name, x)

In [51]:
f = Foo('urkk')

In [53]:
f.spam

<bound method Foo.spam of <__main__.Foo object at 0x00000000054D78D0>>

特性可以截取操作权, 以设置和删除属性;   
向特性附加setter和deleter方法;

In [4]:
KK1

'文档活动位'

In [74]:
class Foo(object):
    def __init__(self ,name):
        self.__name = name
    @property
    def name(self):
        return self.__name
    @name.setter
    def name(self,value):
        if not isinstance(value,str):
            raise TypeError("Must be a string!")
        self.__name = value
    @name.deleter
    def name(self):
        raise TypeError("Can't delete name")

In [75]:
f = Foo('urkk')
f.name

'urkk'

In [76]:
f.name = "urkk2"
f.name

'urkk2'

In [79]:
try:
    del f.name 
except TypeError as e:
    print("TypeError:", e)

TypeError: Can't delete name


## 描述符

## 数据封装和私有属性

## 对象内存管理

## 对象表示和属性绑定

## `__slots__`

## 运算符重载和

## 类型和类成员属性

## 抽象基类

## 原类

## 类型装饰